### LangChain for LLM Application development

### Chains in LangChain

The Chain combines the LLM model together with a prompt and with this building block, you can put a bunch of these building blocks together to carry out a sequence on your text or on your other data.

#### LLMChain

In [ ]:
#%pip install pandas
import pandas as pd
from langchain_community.chat_models import ChatOllama
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

llm_model = "llama3:8b"

llm = ChatOllama(temperature=0.0, model=llm_model) #Adjusting the temperature can make the descriptions more 'fun'

In [34]:
df = pd.read_csv('Data.csv')
df

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...
5,L'Or Espresso Café \n,Je trouve le goût médiocre. La mousse ne tient...
6,Hervidor de Agua Eléctrico,"Está lu bonita calienta muy rápido, es muy fun..."


In [12]:
prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
        a company that make {product}"
)

chain = LLMChain(llm=llm, prompt=prompt)

C:\Users\zanes\AppData\Local\Temp\ipykernel_72500\3537221364.py:6: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt)


In [13]:
product = "Queen Size Sheet Set"
chain.run(product)

C:\Users\zanes\AppData\Local\Temp\ipykernel_72500\550859008.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain.run(product)


"Here are some suggestions for a company name that makes queen-size sheet sets:\n\n1. **Regal Bedding Co.**: This name plays off the idea of a queen-sized bed and implies a sense of luxury and quality.\n2. **Queenly Sheets**: Simple and straightforward, this name highlights the product's focus on queen-size sheets.\n3. **DreamWeave**: This name evokes the idea of cozy, comfortable bedding that helps you dream big.\n4. **Royal Rest**: This name conveys a sense of relaxation and rejuvenation, perfect for a company that makes high-quality bed linens.\n5. **SoftSpun**: This name emphasizes the softness and comfort of the sheets, which is a key selling point for many consumers.\n6. **Bedding Bliss**: This name suggests that the company's products will bring joy and satisfaction to customers' bedrooms.\n7. **Queenly Comforts**: This name combines the idea of queen-size bedding with the promise of comfort and relaxation.\n8. **SlumberCraft**: This name emphasizes the craftsmanship and attenti

#### SimpleSequentialChain

**The idea of sequential chains are to combine multiple chains were the output of the one chain is the input of the next chain**.  

SimpleSequentialChain: Single Input/Output  
SequentialChain: Multiple Input/Outputs

In [14]:
from langchain.chains import SimpleSequentialChain

In [24]:
# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)


In [25]:
# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    company:{company_name}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)


In [27]:

overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two], verbose=True)
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
Here are some suggestions for a company name that specializes in queen-size sheet sets:

1. **Regal Bedding Co.**: This name plays off the idea of a queen-sized bed and implies a sense of luxury and quality.
2. **Queenly Sheets**: Simple and straightforward, this name highlights the product focus on queen-size sheets.
3. **DreamWeave**: This name evokes a sense of comfort and relaxation, suggesting that your sheet sets will help customers have sweet dreams.
4. **SoftTouch Bedding**: This name emphasizes the softness and comfort of your sheet sets, which is likely to be a key selling point for customers.
5. **Royal Rest**: This name positions your company as a provider of high-quality bedding that helps customers get a good night's rest, fit for royalty.
6. **SlumberCraft Co.**: This name suggests a focus on craftsmanship and attention to detail in the production of your sheet sets.
7. **Queenly Quilts & Sheets**: If you also offer quilts 

'Here is a 20-word description for Regal Bedding Co.:\n\n"Regal Bedding Co. crafts luxurious queen-size sheet sets, prioritizing quality, comfort, and style for the perfect night\'s rest."'

#### SequentialChain

In [28]:
from langchain.chains import SequentialChain


In [ ]:
# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:"
    "\n\n{Review}" #Variable passed in at the start
)
# chain 1: input= Review and output= English_Review
chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="English_Review" #Output
                    )

# prompt template 2: summarize the review
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}" #Output of the previous chain is the input for this chain
)
# chain 2: input= English_Review and output= summary
chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="summary"
                    )

# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="language"
                      )

# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )

In [ ]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four], #All teh chains
    input_variables=["Review"], #The human input
    output_variables=["English_Review", "summary","language","followup_message"], #Change to get the answers to the different outputs
    verbose=True
)

In [33]:
review = df.Review[5]
overall_chain(review)



> Entering new SequentialChain chain...

> Finished chain.


{'Review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?",
 'English_Review': 'Here\'s the translation:\n\nI find the taste mediocre. The foam doesn\'t hold up, it\'s weird. I buy the same ones in stores and the taste is much better...\n\nOld batch or counterfeit!?\n\n(Note: "Vieux lot" is a French idiomatic expression that roughly translates to "old batch" or "stale stock", implying that the product may be past its expiration date or not fresh. The reviewer is suggesting that the product might be old or fake.)',
 'summary': "The reviewer finds the taste of this product mediocre and the foam doesn't hold up, suspecting it may be an old or counterfeit batch rather than a genuine one.",
 'language': 'The language of this review is French.',
 'followup_message': "Réponse suivante :\n\nJe suis en désaccord avec l'avis du réviseur quant à la qualité de ce produit. Selon moi

#### RouterChain

This is used to *Route* an input to a chain based on what that input is.

**If you have multiple sub-chains that specialize each in a particular type of input, then you can use a RouterChain that first decides which sub-chain to pass it two and then passes it to that chain.**

In [35]:
# Define different prompt templates for answering different subjects
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [39]:
# The router chain decides when to use what sub-chain based on the prompt information below. Give a name and description
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

In [37]:
from langchain.chains.router import MultiPromptChain #Chain that is used when routing between different templates
#LLMRouterChain uses the LLM itself to route between the different chains
#RouterOutputParser parses the output into a dictionary that can be used downstream to determine which chain to use and the input to that chain
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.prompts import PromptTemplate

In [40]:
#Now we create the destination chains. These are the chains that will be called by the router
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [ ]:
#In addition to the destination chain, we also need a default chain
#This chain is called when the router cannot decide which chains to use
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm,prompt=default_prompt)

In [ ]:
#Now we define the template that is used by the LLM to route between the different chains
#This has instructions of the tasks to be done and also teh specific formatting 
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ "DEFAULT" or name of the prompt to use in {destinations}
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: The value of “destination” MUST match one of \
the candidate prompts listed below.\
If “destination” does not fit any of the specified prompts, set it to “DEFAULT.”
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

<>:2: SyntaxWarning: invalid escape sequence '\ '
<>:2: SyntaxWarning: invalid escape sequence '\ '
C:\Users\zanes\AppData\Local\Temp\ipykernel_72500\1293037028.py:2: SyntaxWarning: invalid escape sequence '\ '
  MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \


In [44]:
#Destinations referring to the different type of prompt infos given such as maths, computer science etc.
#The template is flexible and you can add in more destinations if need be
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)

router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(), #Helps the LLM decide between which sub-chains to route into
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [45]:
chain = MultiPromptChain(router_chain=router_chain,
                         destination_chains=destination_chains,
                         default_chain=default_chain, verbose=True)

C:\Users\zanes\AppData\Local\Temp\ipykernel_72500\1481339893.py:1: LangChainDeprecationWarning: Please see migration guide here for recommended implementation: https://python.langchain.com/docs/versions/migrating_chains/multi_prompt_chain/
  chain = MultiPromptChain(router_chain=router_chain,


In [46]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation in the context of thermodynamics?'}
> Finished chain.


"Blackbody radiation! A fundamental concept in thermodynamics, and one that has far-reaching implications for our understanding of heat transfer and energy exchange.\n\nIn the context of thermodynamics, a blackbody is an idealized object that absorbs all incident electromagnetic radiation (light) without reflecting or transmitting any of it. In other words, it's a perfect absorber of light. This means that when you shine light on a blackbody, it will absorb all the energy and convert it into heat.\n\nNow, when we talk about blackbody radiation, we're referring to the thermal radiation emitted by this idealized object. Since it absorbs all incident radiation, a blackbody will also emit radiation in response to its temperature. This is known as thermal radiation or blackbody radiation.\n\nThe key characteristic of blackbody radiation is that it's a perfect radiator – it emits energy at every possible wavelength (or frequency) across the electromagnetic spectrum, depending on its temperat

In [48]:
chain.run("What is Machine Learning?")



> Entering new MultiPromptChain chain...
computer science: {'input': 'What is Machine Learning? (in the context of AI and data analysis)'}
> Finished chain.


"I'm excited to dive into this topic!\n\nMachine learning (ML) is a subfield of artificial intelligence (AI) that enables computers to learn from data without being explicitly programmed. In other words, ML allows systems to improve their performance on a task over time by automatically adjusting their behavior based on the data they receive.\n\nHere's how it works:\n\n1. **Data Collection**: You gather a dataset related to the problem you want to solve. This can be images, text, audio, or any other type of data.\n2. **Model Training**: You train a machine learning model using this dataset. The model is essentially a set of algorithms that analyze the data and learn patterns, relationships, and correlations within it.\n3. **Model Evaluation**: You evaluate the performance of your trained model on a separate test dataset to ensure it generalizes well and doesn't overfit (i.e., performs poorly on new, unseen data).\n4. **Deployment**: Once you're satisfied with the model's performance, y